<a href="https://colab.research.google.com/github/taciga30/SQL-Murder-Mystery/blob/Soluci%C3%B3n-Caso-de-Asesinato/Proyecto_1_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [ ]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [ ]:
%%sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name ASC, column_name ASC

,table_name,column_name,data_type
0,crime_scene_report,city,VARCHAR
1,crime_scene_report,date,BIGINT
2,crime_scene_report,description,VARCHAR
3,crime_scene_report,type,VARCHAR
4,drivers_license,age,BIGINT
5,drivers_license,car_make,VARCHAR
6,drivers_license,car_model,VARCHAR
7,drivers_license,eye_color,VARCHAR
8,drivers_license,gender,VARCHAR
9,drivers_license,hair_color,VARCHAR


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

In [ ]:
%%sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name ASC, column_name ASC

,table_name,column_name,data_type
0,crime_scene_report,city,VARCHAR
1,crime_scene_report,date,BIGINT
2,crime_scene_report,description,VARCHAR
3,crime_scene_report,type,VARCHAR
4,drivers_license,age,BIGINT
5,drivers_license,car_make,VARCHAR
6,drivers_license,car_model,VARCHAR
7,drivers_license,eye_color,VARCHAR
8,drivers_license,gender,VARCHAR
9,drivers_license,hair_color,VARCHAR


___

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

# **INVESTIGACIÓN** 🕵

---



### **PASO 1**
Verificar las tablas disponibles de la base de datos del Departamento de la Policia.

In [ ]:
%%sql
SELECT table_name
FROM information_schema.tables

,table_name
0,crime_scene_report
1,drivers_license
2,facebook_event_checkin
3,get_fit_now_check_in
4,get_fit_now_member
5,income
6,interview
7,person


### **PASO 2**
Consultar la información en la tabla ***crime_scene_report*** para verificar los crímenes de tipo asesinato en la fecha en que ocurrió el asesinato y en la ubicación específica.

In [ ]:
%%sql
SELECT *
FROM crime_scene_report
WHERE date=20180115 AND city ='SQL City' AND type = 'murder'


,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


### **PASO 3**
Tomar la información del reporte que se tiene del caso:  

> "*Las imágenes de seguridad muestran que hubo 2 testigos. El primer testigo vive en la última casa en "Northwestern Dr". El segundo testigo, llamado Annabel, vive en algún lugar de "Franklin Ave".*"

para indagar que dijeron los testigos.

In [ ]:
%%sql
SELECT MAX(address_number)
FROM person
WHERE
  address_street_name='Northwestern Dr'

,max(address_number)
0,4919


In [ ]:
%%sql
SELECT *
FROM person
WHERE
  address_street_name LIKE 'Northwestern Dr' AND
  address_number = 4919

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
%%sql
SELECT *
FROM person
WHERE
  name LIKE 'Annabel%' AND
  address_street_name LIKE 'Franklin Ave'


,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


### **PASO 4**
Con la información de los testigos se verifica que dijeron en sus entrevistas.

In [ ]:
%%sql
SELECT person_id, name, transcript
FROM interview JOIN person ON id=person_id
WHERE person_id IN(14887, 16371)

,person_id,name,transcript
0,14887,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."
1,16371,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."


### **PASO 5**
De acuerdo al testimonio de ***Morty Schapiro:***
> *Escuché un disparo y luego vi a un hombre salir corriendo. Tenía una bolsa de "Get Fit Now Gym". El número de membresía en la bolsa comenzaba con "48Z". Sólo los miembros de oro tienen esas bolsas. El hombre se subió a un coche con una placa que decía "H42W".*

se verifican la tabla de membresias del Gimnasio para identificar los miembros gold y que su membresia empiece por "48Z".

Con esta información se consulta la tabla de personas para verificar el número de licencia de los sospechosos y así contrastarlos con las coincidencias de las placas que contienen "H42W"


In [ ]:
%%sql
SELECT *
FROM get_fit_now_member
WHERE id LIKE '48Z%' AND membership_status LIKE 'gold'

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [ ]:
%%sql
SELECT *
FROM person
WHERE name LIKE 'Joe Germuska' OR name LIKE 'Jeremy Bowers'

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [ ]:
%%sql
SELECT *
FROM drivers_license
WHERE gender LIKE 'male'
  AND plate_number LIKE '%H42W%'
  AND id IN (173289, 423327)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


### **PASO 6**
Y de acuerdo al testimonio de ***Annabel Miller:***
> *Vi ocurrir el asesinato y reconocí al asesino en mi gimnasio cuando estaba haciendo ejercicio la semana pasada, el 9 de enero.*

se consulta la tabla de registros de ingreso del gimnasio del 9 de enero para confirmar la información y verificar que el nombre arrojado como el asesino tambien se encuentre entre la información dada por la testigo.

In [ ]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%' AND check_in_date= 20180109

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


### **PASO 7**

Validar la entrevista del acusado ***Jeremy Bowers***

In [ ]:
%%sql
SELECT person_id, name, transcript
FROM interview JOIN person ON id=person_id
WHERE person_id = 67318

,person_id,name,transcript
0,67318,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."


### **PASO 8**
Según la declaración del acusado ***Jeremy Bowers***:

> *Me contrató una mujer con mucho dinero. No sé su nombre, pero sé que mide alrededor de 5'5" (65") o 5'7" (67"). Tiene el pelo rojo y conduce un Tesla Model S. Sé que asistió al Concierto Sinfónico SQL tres veces en diciembre de 2017*.

Se verifica la información dada y se consulta la tabla de vehículos para encontrar el dueño del vehículo.





In [ ]:
%%sql
SELECT *
FROM drivers_license
WHERE gender LIKE 'female'
  AND car_make	LIKE 'Tesla'
  AND car_model LIKE 'Model S'
  AND hair_color LIKE 'red'
  AND height BETWEEN 65 AND 67

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


Ahora se verifica la información de los eventos para identificar quien coincide que asistió a el Concierto Sinfónico SQL tres veces en diciembre de 2017

In [ ]:
%%sql
SELECT person_id, event_id, event_name, COUNT(DISTINCT date) AS 'Veces en el evento', license_id
FROM facebook_event_checkin
JOIN person ON facebook_event_checkin.person_id = person.id
WHERE event_name LIKE 'SQL Symphony Concert'
  AND date BETWEEN 20171201 AND 20171231
GROUP BY person_id, event_id, event_name, license_id
HAVING COUNT(DISTINCT date) = 3;

,person_id,event_id,event_name,Veces en el evento,license_id
0,24556,1143,SQL Symphony Concert,3,101191
1,99716,1143,SQL Symphony Concert,3,202298


Se cruzan ambas consultas para identificar quién es la asesina intelectual de acuerdo con la información que dió el asesino arrestado.

In [ ]:
%%sql
SELECT f.person_id, p.name, f.event_id, f.event_name, COUNT(DISTINCT f.date) AS 'Veces en el evento', p.license_id, d.gender, d.car_make, d.car_model, d.hair_color, d.height
FROM facebook_event_checkin f
JOIN person p ON f.person_id = p.id
JOIN drivers_license d ON p.license_id = d.id
WHERE f.event_name LIKE 'SQL Symphony Concert'
  AND f.date BETWEEN 20171201 AND 20171231
  AND d.gender LIKE 'female'
  AND d.car_make LIKE 'Tesla'
  AND d.car_model LIKE 'Model S'
  AND d.hair_color LIKE 'red'
  AND d.height BETWEEN 65 AND 67
GROUP BY f.person_id, p.name, f.event_id, f.event_name, p.license_id, d.gender, d.car_make, d.car_model, d.hair_color, d.height
HAVING COUNT('Veces en el evento') = 3

,person_id,name,event_id,event_name,Veces en el evento,license_id,gender,car_make,car_model,hair_color,height
0,99716,Miranda Priestly,1143,SQL Symphony Concert,3,202298,female,Tesla,Model S,red,66


## Resolviendo el Misterio

Siguiendo las pistas en la base de datos del Departamento de Policía para encontrar al asesino, se logra dar con el asesino material y el asesino intelectual de este caso.



In [ ]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

In [ ]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True